In [2]:
import pconbr
import pconbr.bench
import pconbr.bench.count
import pconbr.identity
import pconbr.kmer_count
import pconbr.kmer_count.curve

# Code name: pconbr

Project target: Use a kmer counter to perform a pre-correction step on long-read data

## Dataset

### References

| code name       | species         | path                          | genome size | 
|:----------------|:----------------|:------------------------------|------------:|
| s_pneumoniae    | S. pneumoniae   | referencesCP026549.fasta      |      2.2 Mb |
| c_vartiovaarae  | C. vartiovaarae |                               |     ~11.2Mb |
| e_coli_ont      | E. coli         | references/CP028309.fasta      |       4.7Mb |
| e_coli_pb       | E. coli         | references/CP028309.fasta      |       4.7Mb |
| s_cerevisiae    | S. cerevisiae   | references/GCA_002163515.fasta |      12.4Mb |


### Reads
| code name       | species         | path                        | # bases (Gb)| coverage |
|:----------------|:----------------|:----------------------------|------------:|---------:|
| s_pneumoniae    | S. pneumoniae   | reads/SRR8556426.fasta      |         2.2 |   ~1000x |
| c_vartiovaarae  | C. vartiovaarae | reads/ERR18779[66-70].fasta |         1.7 |    ~150x |
| e_coli_ont      | E. coli         | reads/SRR8494940.fasta      |         1.6 |    ~340x |
| e_coli_pb       | E. coli         | reads/SRR8494911.fasta      |         1.4 |    ~297x |
| s_cerevisiae    | S. cerevisiae   | reads/SRR2157264_[1-2]      |       0.187 |     ~15x |



In [2]:
# To download reference genome uncomment next line and execute this cell can take many time
#!./script/dl_ref.sh

In [3]:
# To download data uncomment next line and execute this cell can take many time
#!./script/dl_reads.sh

## Kmer counting

In [4]:
# To perform pcon kmc and jellyfish count on dataset uncomment next line and execute this cell
#!snakemake -s pipeline/count.snakefile count_all

File benchmark/{counter name}/{dataset codename}.tsv contains time (in second) and memory (in Mb) usage of each run this information was resume in this table.

In [5]:
pconbr.bench.count.get("time")

jellyfish       kmc      pcon
dataset        k                                 
c_vartiovaarae k13   413.5882  193.6503   53.1597
               k15   766.9468  784.6563   63.4619
               k17  1288.2844  833.9791  252.0244
e_coli_ont     k13   403.0423  184.5209   49.9495
               k15  1135.5411  724.3784   61.9082
               k17  1166.9966  780.5869  214.6334
e_coli_pb      k13   361.0433  157.6295   44.4903
               k15  1456.8800  732.9156   54.2674
               k17  1255.0258  766.6578  215.9037
s_cerevisiae   k13    56.8721   24.2400    6.6724
               k15   123.3701  130.9554    9.8227
               k17   290.8972  133.2776   63.6876
s_pneumoniae   k13   540.4675  265.6703   66.9792
               k15   905.0415  870.6088   84.3198
               k17  1301.9399  939.4879  284.0073

In [6]:
pconbr.bench.count.get("memory")

jellyfish       kmc     pcon
dataset        k                               
c_vartiovaarae k13   1581.97   2143.56    22.00
               k15   6204.13  10830.66   262.54
               k17  16391.34  11046.30  4101.01
e_coli_ont     k13   1387.03   2219.09    22.03
               k15  22121.59  10636.99   262.75
               k17  16391.30  10898.98  4103.75
e_coli_pb      k13   1993.30   2034.21    21.91
               k15  35264.41  10717.47   262.36
               k17  16391.14  11079.68  4103.34
s_cerevisiae   k13    257.59    657.64    22.02
               k15   1957.95   1442.41   262.57
               k17  16391.10   1367.47  4092.84
s_pneumoniae   k13   1783.90   2810.83    21.98
               k15  11002.25  11242.54   262.57
               k17  16391.33  11273.15  4103.57

## PconBr on simulated dataset evaluation


Error rate was evaluate by `samtools stats` line `error rate:`.

Read was simulate by [Badread](https://github.com/rrwick/Badread) on E. coli CFT073 genome ([ENA id CP028309](https://www.ebi.ac.uk/ena/data/view/CP028309)), error rate 5.625682.

We evaluate identity before pconbr pipeline with diffrente value of kmer size (k), br method.

In [7]:
# Run some snakemake pipeline to test parameter on dataset
#!snakemake -s pipeline/all.snakefile pconbr_eval

### Synthetic dataset 95

Error rate:

In [3]:
float(pconbr.identity.get_error_rate("reads/simulated_reads_95.stats"))*100

FileNotFoundError: [Errno 2] No such file or directory: 'reads/simulated_reads_95.stats'

#### Kmer spectrum

In [4]:
#!snakemake -s pipeline/generate_stat.snakefile kmer_spectrum_simulated_reads
import pandas
import plotly.graph_objects as go

df_true_false = pandas.read_csv("stats/kmer_spectrum/simulated_reads_95_true_false.csv", index_col=0)
df_all = pandas.read_csv("stats/kmer_spectrum/simulated_reads_95.csv", index_col=0)

df = pandas.merge(df_true_false, df_all, left_index=True, right_index=True)


fig = go.Figure(data=[go.Bar(name=c, y=df[c]) for c in df.columns],
               layout=go.Layout(yaxis=dict(range=[0, 275_000]),
                                xaxis=dict(range=[0, 125]))
               )

#fig.update_layout(yaxis_type="log")
fig.show()

ModuleNotFoundError: No module named 'plotly'

#### Correction with genomic kmer

Difference between original error rate and the corrected read error rate

In [9]:
pconbr.identity.genomic_kmer("simulated_reads_95")

,s1,s2,s3,s4,s5,s6,s7,s8,s9
k,,,,,,,,,
9,0.00963,0.00922,0.00889,0.00875,0.00875,0.00875,0.00875,0.00874,0.00874
11,1.39132,1.16851,1.01168,0.90589,0.83365,0.78273,0.74338,0.71065,0.67966
13,11.67877,5.07770,2.00401,0.65655,0.06778,-0.18413,-0.28398,-0.31269,-0.30837
15,0.56856,-1.45660,-1.56729,-1.36834,-1.14115,-0.94642,-0.78921,-0.66178,-0.55848
17,-2.95722,-2.11007,-1.61401,-1.27031,-1.01657,-0.82709,-0.68288,-0.56939,-0.47824
19,-0.01686,-0.00302,-0.00054,-0.00014,-0.00007,-0.00005,-0.00004,-0.00003,-0.00002


#### Correction with reads kmer

Difference between original error rate and the corrected read error rate

In [10]:
pconbr.identity.read_kmer("simulated_reads_95")

s1       s2       s3       s4       s5       s6       s7       s8  \
a  k                                                                            
1  13  0.00001  0.00001  0.00001  0.00001  0.00001  0.00001  0.00001  0.00001   
   15  0.00067  0.00076  0.00073  0.00061  0.00050  0.00041  0.00031  0.00023   
   17  0.00076  0.00056  0.00015 -0.00006 -0.00014 -0.00015 -0.00014 -0.00012   
   19 -0.00003 -0.00031 -0.00036 -0.00033 -0.00029 -0.00025 -0.00021 -0.00017   
2  13  0.04044  0.03584  0.03267  0.03075  0.02955  0.02883  0.02846  0.02826   
   15  6.55711  3.31451  1.71992  0.96090  0.57859  0.36893  0.24615  0.16933   
   17  0.42643 -0.39124 -0.49705 -0.45951 -0.39870 -0.34152 -0.29237 -0.25137   
   19 -0.87780 -0.74970 -0.60991 -0.49869 -0.41140 -0.34309 -0.28894 -0.24510   
3  13  0.15975  0.13760  0.12353  0.11487  0.10957  0.10640  0.10444  0.10330   
   15  6.09556  2.70900  1.20231  0.54095  0.23492  0.08442  0.00693 -0.03386   
   17 -0.99509 -1.08049 -0.92999 -0.77377 -0.64147 -0.53540 -0.45027 -0.38152   
   19 -1.59294 -1.15576 -0.89200 -0.71007 -0.57479 -0.47214 -0.39231 -0.32885   
4  13  0.35322  0.29815  0.26419  0.24353  0.23054  0.22227  0.21696  0.21358   
   15  4.70260  1.66647  0.48549  0.03400 -0.13913 -0.20216 -0.21853 -0.21510   
   17 -1.75095 -1.46258 -1.17440 -0.94814 -0.77187 -0.63562 -0.52914 -0.44425   
   19 -1.88160 -1.28371 -0.96728 -0.75861 -0.60634 -0.49252 -0.40577 -0.33736   
5  13  0.60302  0.49870  0.43631  0.39763  0.37280  0.35684  0.34638  0.33931   
   15  3.46699  0.78069 -0.09394 -0.36200 -0.42263 -0.41363 -0.38132 -0.34270   
   17 -2.07648 -1.60689 -1.25692 -0.99987 -0.80570 -0.65782 -0.54371 -0.45362   
   19 -1.80685 -1.21420 -0.90662 -0.70470 -0.55866 -0.45041 -0.36832 -0.30397   
6  13  0.89057  0.72394  0.62403  0.56223  0.52231  0.49618  0.47876  0.46673   
   15  2.56173  0.14150 -0.49970 -0.63143 -0.61108 -0.55099 -0.48419 -0.42186   
   17 -2.09328 -1.58447 -1.22595 -0.96768 -0.77448 -0.62844 -0.51663 -0.42849   
   19 -1.55554 -1.04772 -0.77910 -0.60164 -0.47334 -0.37858 -0.30717 -0.25133   
7  13  1.20171  0.95850  0.81446  0.72508  0.66730  0.62913  0.60333  0.58507   
   15  1.95062 -0.28348 -0.76122 -0.79921 -0.72421 -0.62962 -0.54119 -0.46375   
   17 -1.93159 -1.45967 -1.12507 -0.88331 -0.70309 -0.56728 -0.46368 -0.38232   
   19 -1.25708 -0.85249 -0.63147 -0.48428 -0.37795 -0.29967 -0.24095 -0.19535   
8  13  1.52539  1.19173  0.99782  0.87743  0.80010  0.74862  0.71314  0.68757   
   15  1.56940 -0.53708 -0.90546 -0.88397 -0.77530 -0.66120 -0.56070 -0.47537   
   17 -1.68242 -1.28307 -0.98720 -0.77107 -0.61000 -0.48904 -0.39708 -0.32531   
   19 -0.97337 -0.66360 -0.48939 -0.37241 -0.28802 -0.22602 -0.17987 -0.14427   
9  13  1.85171  1.41747  1.16881  1.01543  0.91705  0.85090  0.80519  0.77177   
   15  1.34770 -0.66329 -0.96263 -0.90568 -0.77983 -0.65675 -0.55161 -0.46396   
   17 -1.40774 -1.08681 -0.83463 -0.64861 -0.50984 -0.40575 -0.32701 -0.26575   
   19 -0.72746 -0.49789 -0.36461 -0.27480 -0.21011 -0.16283 -0.12812 -0.10137   
10 13  2.18375  1.63559  1.32639  1.13831  1.01816  0.93710  0.88028  0.83853   
   15  1.23440 -0.69816 -0.95552 -0.88204 -0.75015 -0.62593 -0.52163 -0.43555   
   17 -1.13835 -0.89026 -0.68220 -0.52697 -0.41113 -0.32451 -0.25933 -0.20870   
   19 -0.52908 -0.36206 -0.26299 -0.19601 -0.14800 -0.11317 -0.08785 -0.06852   
11 13  2.51054  1.83888  1.46595  1.24303  1.10060  1.00480  0.93700  0.88678   
   15  1.18269 -0.66986 -0.90276 -0.82433 -0.69497 -0.57551 -0.47627 -0.39500   
   17 -0.89658 -0.70952 -0.54200 -0.41550 -0.32118 -0.25106 -0.19894 -0.15845   
   19 -0.37546 -0.25572 -0.18388 -0.13507 -0.10070 -0.07597 -0.05814 -0.04475   
12 13  2.83965  2.03570  1.59619  1.33624  1.17070  1.05945  0.98064  0.92166   
   15  1.16184 -0.60250 -0.82136 -0.74612 -0.62475 -0.51363 -0.42201 -0.34744   
   17 -0.68984 -0.55194 -0.41986 -0.31926 -0.24433 -0.18894 -0.14797 -0.11660   
   19 -0.26174 -0.17726 -0.12583 -0.09120 -

### Synthetic dataset 96

Error rate:

In [3]:
float(pconbr.identity.get_error_rate("reads/simulated_reads_96.stats"))*100

FileNotFoundError: [Errno 2] No such file or directory: 'reads/simulated_reads_95.stats'

#### Kmer spectrum

In [4]:
#!snakemake -s pipeline/generate_stat.snakefile kmer_spectrum_simulated_reads
import pandas
import plotly.graph_objects as go

df_true_false = pandas.read_csv("stats/kmer_spectrum/simulated_reads_95_true_false.csv", index_col=0)
df_all = pandas.read_csv("stats/kmer_spectrum/simulated_reads_95.csv", index_col=0)

df = pandas.merge(df_true_false, df_all, left_index=True, right_index=True)


fig = go.Figure(data=[go.Bar(name=c, y=df[c]) for c in df.columns],
               layout=go.Layout(yaxis=dict(range=[0, 275_000]),
                                xaxis=dict(range=[0, 125]))
               )

#fig.update_layout(yaxis_type="log")
fig.show()

ModuleNotFoundError: No module named 'plotly'

#### Correction with genomic kmer

Difference between original error rate and the corrected read error rate

In [9]:
pconbr.identity.genomic_kmer("simulated_reads_95")

,s1,s2,s3,s4,s5,s6,s7,s8,s9
k,,,,,,,,,
9,0.00963,0.00922,0.00889,0.00875,0.00875,0.00875,0.00875,0.00874,0.00874
11,1.39132,1.16851,1.01168,0.90589,0.83365,0.78273,0.74338,0.71065,0.67966
13,11.67877,5.07770,2.00401,0.65655,0.06778,-0.18413,-0.28398,-0.31269,-0.30837
15,0.56856,-1.45660,-1.56729,-1.36834,-1.14115,-0.94642,-0.78921,-0.66178,-0.55848
17,-2.95722,-2.11007,-1.61401,-1.27031,-1.01657,-0.82709,-0.68288,-0.56939,-0.47824
19,-0.01686,-0.00302,-0.00054,-0.00014,-0.00007,-0.00005,-0.00004,-0.00003,-0.00002


#### Correction with reads kmer

Difference between original error rate and the corrected read error rate

In [10]:
pconbr.identity.read_kmer("simulated_reads_95")

s1       s2       s3       s4       s5       s6       s7       s8  \
a  k                                                                            
1  13  0.00001  0.00001  0.00001  0.00001  0.00001  0.00001  0.00001  0.00001   
   15  0.00067  0.00076  0.00073  0.00061  0.00050  0.00041  0.00031  0.00023   
   17  0.00076  0.00056  0.00015 -0.00006 -0.00014 -0.00015 -0.00014 -0.00012   
   19 -0.00003 -0.00031 -0.00036 -0.00033 -0.00029 -0.00025 -0.00021 -0.00017   
2  13  0.04044  0.03584  0.03267  0.03075  0.02955  0.02883  0.02846  0.02826   
   15  6.55711  3.31451  1.71992  0.96090  0.57859  0.36893  0.24615  0.16933   
   17  0.42643 -0.39124 -0.49705 -0.45951 -0.39870 -0.34152 -0.29237 -0.25137   
   19 -0.87780 -0.74970 -0.60991 -0.49869 -0.41140 -0.34309 -0.28894 -0.24510   
3  13  0.15975  0.13760  0.12353  0.11487  0.10957  0.10640  0.10444  0.10330   
   15  6.09556  2.70900  1.20231  0.54095  0.23492  0.08442  0.00693 -0.03386   
   17 -0.99509 -1.08049 -0.92999 -0.77377 -0.64147 -0.53540 -0.45027 -0.38152   
   19 -1.59294 -1.15576 -0.89200 -0.71007 -0.57479 -0.47214 -0.39231 -0.32885   
4  13  0.35322  0.29815  0.26419  0.24353  0.23054  0.22227  0.21696  0.21358   
   15  4.70260  1.66647  0.48549  0.03400 -0.13913 -0.20216 -0.21853 -0.21510   
   17 -1.75095 -1.46258 -1.17440 -0.94814 -0.77187 -0.63562 -0.52914 -0.44425   
   19 -1.88160 -1.28371 -0.96728 -0.75861 -0.60634 -0.49252 -0.40577 -0.33736   
5  13  0.60302  0.49870  0.43631  0.39763  0.37280  0.35684  0.34638  0.33931   
   15  3.46699  0.78069 -0.09394 -0.36200 -0.42263 -0.41363 -0.38132 -0.34270   
   17 -2.07648 -1.60689 -1.25692 -0.99987 -0.80570 -0.65782 -0.54371 -0.45362   
   19 -1.80685 -1.21420 -0.90662 -0.70470 -0.55866 -0.45041 -0.36832 -0.30397   
6  13  0.89057  0.72394  0.62403  0.56223  0.52231  0.49618  0.47876  0.46673   
   15  2.56173  0.14150 -0.49970 -0.63143 -0.61108 -0.55099 -0.48419 -0.42186   
   17 -2.09328 -1.58447 -1.22595 -0.96768 -0.77448 -0.62844 -0.51663 -0.42849   
   19 -1.55554 -1.04772 -0.77910 -0.60164 -0.47334 -0.37858 -0.30717 -0.25133   
7  13  1.20171  0.95850  0.81446  0.72508  0.66730  0.62913  0.60333  0.58507   
   15  1.95062 -0.28348 -0.76122 -0.79921 -0.72421 -0.62962 -0.54119 -0.46375   
   17 -1.93159 -1.45967 -1.12507 -0.88331 -0.70309 -0.56728 -0.46368 -0.38232   
   19 -1.25708 -0.85249 -0.63147 -0.48428 -0.37795 -0.29967 -0.24095 -0.19535   
8  13  1.52539  1.19173  0.99782  0.87743  0.80010  0.74862  0.71314  0.68757   
   15  1.56940 -0.53708 -0.90546 -0.88397 -0.77530 -0.66120 -0.56070 -0.47537   
   17 -1.68242 -1.28307 -0.98720 -0.77107 -0.61000 -0.48904 -0.39708 -0.32531   
   19 -0.97337 -0.66360 -0.48939 -0.37241 -0.28802 -0.22602 -0.17987 -0.14427   
9  13  1.85171  1.41747  1.16881  1.01543  0.91705  0.85090  0.80519  0.77177   
   15  1.34770 -0.66329 -0.96263 -0.90568 -0.77983 -0.65675 -0.55161 -0.46396   
   17 -1.40774 -1.08681 -0.83463 -0.64861 -0.50984 -0.40575 -0.32701 -0.26575   
   19 -0.72746 -0.49789 -0.36461 -0.27480 -0.21011 -0.16283 -0.12812 -0.10137   
10 13  2.18375  1.63559  1.32639  1.13831  1.01816  0.93710  0.88028  0.83853   
   15  1.23440 -0.69816 -0.95552 -0.88204 -0.75015 -0.62593 -0.52163 -0.43555   
   17 -1.13835 -0.89026 -0.68220 -0.52697 -0.41113 -0.32451 -0.25933 -0.20870   
   19 -0.52908 -0.36206 -0.26299 -0.19601 -0.14800 -0.11317 -0.08785 -0.06852   
11 13  2.51054  1.83888  1.46595  1.24303  1.10060  1.00480  0.93700  0.88678   
   15  1.18269 -0.66986 -0.90276 -0.82433 -0.69497 -0.57551 -0.47627 -0.39500   
   17 -0.89658 -0.70952 -0.54200 -0.41550 -0.32118 -0.25106 -0.19894 -0.15845   
   19 -0.37546 -0.25572 -0.18388 -0.13507 -0.10070 -0.07597 -0.05814 -0.04475   
12 13  2.83965  2.03570  1.59619  1.33624  1.17070  1.05945  0.98064  0.92166   
   15  1.16184 -0.60250 -0.82136 -0.74612 -0.62475 -0.51363 -0.42201 -0.34744   
   17 -0.68984 -0.55194 -0.41986 -0.31926 -0.24433 -0.18894 -0.14797 -0.11660   
   19 -0.26174 -0.17726 -0.12583 -0.09120 -

## Long read correction

To evaluate our correction against other tools we : 
- result against reference genome we use [ELECTOR](//doi.org/10.1101/512889) 
- assembly result (redbean, rala, flye) we use [QUAST](//doi.org/10.1093/bioinformatics/bty266)

### Self correction

We compare pconbr against other self correction tools.

| Tools name | Reference                                                                |
|:-----------|:-------------------------------------------------------------------------|
| CONSENT    | [10.1101/546630](//doi.org/10.1101/546630)                               |
| daccord    | [10.1101/106252](//doi.org/10.1101/106252)                               |
| FLAS       | [10.1093/bioinformatics/btz206](//doi.org/10.1093/bioinformatics/btz206) |
| MECAT      | [10.1038/nmeth.4432](//doi.org/10.1038/nmeth.4432)                       |

#### Mapping result

In [28]:
display.Markdown("TODO")

AttributeError: 'function' object has no attribute 'Markdown'

#### Assembly result

In [ ]:
display.Markdown("TODO")

### Hybrid correction

We compare pconbr against other self correction tools.

| Tools name | Reference                                                                |
|:-----------|:-------------------------------------------------------------------------|


#### Mapping result

In [ ]:
display.Markdown("TODO")

##### Assembly result

In [ ]:
display.Markdown("TODO")

## Polishing

